In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os, os.path
import cv2

In [2]:
#Train Images
directory = r'C:\Users\Anqi\Documents\TensorFlow\Train\Avi'
total_files = 0
for filename in os.listdir(directory):
    total_files = total_files + 1

train_images = np.empty([total_files, 228,228,3],dtype = int)
train_labels = np.empty([total_files, 14,2],dtype = int)

labels = np.empty([14,2],dtype = int)
normalized_labels = np.empty([total_files, 28,1],dtype = float)

bw = 228
bh = 228
bc = 114

for i in range(0, total_files):
    train_images[i] = cv2.imread(r'C:\Users\Anqi\Documents\TensorFlow\Train\Avi\Train' + str(i+1) +'.jpg')
    
    with open(r"C:\Users\Anqi\Documents\TensorFlow\Train\Labels\Train" + str(i+1) +".txt", "r") as file_object:
        for j in range(0,14):
            text = file_object.readline()
            x, y = text.split(", ")
            labels[j,0] = x
            labels[j,1] = y
        train_labels[i] = labels
        
        for j in range (0,14):
            normalized_labels[i,2*j] = (train_labels[i,j,0] - bc)/bw
            normalized_labels[i,2*j+1] = (train_labels[i,j,1] - bc)/bw
        
train_images = train_images / 255.0
train_labels = normalized_labels    

labels = np.empty([14,2],dtype = int)
normalized_labels = np.empty([total_files, 28,1],dtype = float)

#Test Images
directory = r'C:\Users\Anqi\Documents\TensorFlow\Train\Test_Avi'
total_files = 0
for filename in os.listdir(directory):
    total_files = total_files + 1

test_images = np.empty([total_files, 228,228,3],dtype = int)
test_labels = np.empty([total_files, 14,2],dtype = int)

labels = np.empty([14,2],dtype = int)
normalized_labels = np.empty([total_files, 28,1],dtype = float)

for i in range(0, total_files):
    test_images[i] = cv2.imread(r'C:\Users\Anqi\Documents\TensorFlow\Train\Test_Avi\Train' + str(i+1) +'.jpg')
    with open(r"C:\Users\Anqi\Documents\TensorFlow\Train\Test_Labels\Train" + str(i+1) +".txt", "r") as file_object:
        for j in range(0,14):
            text = file_object.readline()
            x, y = text.split(", ")
            labels[j,0] = x
            labels[j,1] = y
            test_labels[i] = labels
            
            for j in range (0,14):
                normalized_labels[i,2*j] = (test_labels[i,j,0] - bc)/bw
                normalized_labels[i,2*j+1] = (test_labels[i,j,1] - bc)/bw

test_images = test_images / 255.0
test_labels = normalized_labels    

print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(test_labels.shape)

(370, 228, 228, 3)
(60, 228, 228, 3)
(370, 28, 1)
(60, 28, 1)


In [3]:
#AlexNet Neural Network
model = models.Sequential()

#Convolutions 96 kernals of size 11x11x3
model.add(layers.Conv2D(96, kernel_size = 11, strides = 4, activation='relu', input_shape=(228, 228, 3)))

#Normalizes values using default normalization values
model.add(layers.LayerNormalization(trainable=False))

#Max Pooling 3x3 stride 2
model.add(layers.MaxPooling2D((3, 3),strides = 2))

#Convolutions 256 kernals of size 5x5x48
model.add(layers.Conv2D(256, kernel_size = 5, padding = 'same', activation='relu'))

#Normalizes values using default normalization values
model.add(layers.LayerNormalization(trainable=False))

#Max Pooling 3x3 stride 2
model.add(layers.MaxPooling2D((3, 3),strides = 2))

#Convolutions 384 kernals of size 3x3x256
model.add(layers.Conv2D(384, kernel_size = 3, padding = 'same', activation='relu'))

#Convolutions 384 kernals of size 3x3x192
model.add(layers.Conv2D(384, kernel_size = 3, padding = 'same', activation='relu'))

#Convolutions 256 kernals of size 3x3x192
model.add(layers.Conv2D(256, kernel_size = 3, padding = 'same', activation='relu'))

#Max Pooling 3x3 stride 2
model.add(layers.MaxPooling2D((3, 3),strides = 2))

#Flattens values to 1 dimensional vector
model.add(layers.Flatten())

#Fully connected 4096 
model.add(layers.Dense(4096, activation='relu'))

#Fully connected 4096
model.add(layers.Dense(4096, activation='relu'))

#Output Layer of 28 x and y coordinate values
model.add(layers.Dense(28))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
layer_normalization (LayerNo (None, 55, 55, 96)        192       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
layer_normalization_1 (Layer (None, 27, 27, 256)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [5]:
#Uses mean squared error to backpropogate error
model.compile(loss='mean_squared_error', optimizer='adam')

#Creates convoluted neural network with given training data and test data
history = model.fit(train_images, train_labels, epochs=100, 
                    validation_data=(test_images, test_labels))

Train on 370 samples, validate on 60 samples
Epoch 1/100
370/370 [==============================] - 3s 7ms/sample - loss: 68.1093 - val_loss: 0.0109
Epoch 2/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0442 - val_loss: 0.0134
Epoch 3/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0078 - val_loss: 0.0050
Epoch 4/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0041 - val_loss: 0.0041
Epoch 5/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0039 - val_loss: 0.0040
Epoch 6/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0038 - val_loss: 0.0041
Epoch 7/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0038 - val_loss: 0.0041
Epoch 8/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0038 - val_loss: 0.0042
Epoch 9/100
370/370 [==============================] - 1s 2ms/sample - loss: 0.0039 - val_loss: 0.0041
Epoch 10/100
370/370 [=====

In [18]:
#model.save('model1.h5') 